# Setup

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, RepeatedKFold, cross_val_score
import sklearn.metrics as skm
from sklearn.metrics import accuracy_score
from sklearn.cluster import AffinityPropagation as AP
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from xgboost import XGBRegressor, XGBClassifier

import re
import time
import datetime as dt

import sys
import os
sys.path.append(os.path.abspath
                (os.path.join
                 (os.path.dirname("constants.py"), '..')))
from constants import *
from scripts import *

import warnings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

hide_toggle()

# FUNCTIONS

In [2]:
def train_test_split_binary(dfToSplitFunc, tournamentId, currentTarget, cut_off_var, splitType, verbose=True):
    
    for col in dfToSplitFunc.columns:
            dfToSplitFunc[col] = dfToSplitFunc[col].fillna(0)
    
    if splitType==0:
        testData = dfToSplitFunc[dfToSplitFunc['tournament_id']==tournamentId].copy()
        xtest= testData.drop(['Date',currentTarget],axis=1).copy()
        xtest= xtest.drop(offCols,axis=1,errors='ignore')
        ytest = testData[currentTarget]

        trainData = dfToSplitFunc[dfToSplitFunc['tournament_id']!=tournamentId].copy()
        xtrain = trainData.drop(['Date',currentTarget],axis=1).copy()
        xtrain = xtrain.drop(offCols,axis=1,errors='ignore')
        ytrain = trainData[currentTarget]
        
    elif splitType==1:
        #print(dfToSplitFunc.columns)
        xCols = dfToSplitFunc.drop(['Date', currentTarget]+offCols,axis=1,errors='ignore')
        #print(xCols.columns)
        yCols = dfToSplitFunc[currentTarget]
        xtrain, xtest, ytrain, ytest = train_test_split(xCols, yCols, test_size=0.20, shuffle=False)
    
    if list(ytrain).count(0)/len(ytrain)==1:
        print(len(ytrain))
        print(dropTypeF)
        print(dfToSplitFunc[currentTarget])
        print('==========================================================')
    
    ytrain_mean, ytrain_std = np.mean(ytrain), np.std(ytrain)
    cut_off = ytrain_std * cut_off_var
    lower, upper = ytrain_mean - cut_off, ytrain_mean + cut_off
    
    outlierMask = ytrain.apply(lambda x: False if x < lower or x > upper else True)
    
    if verbose:
        print(f'train len: {len(xtrain)}')
    lentemp = len(xtrain)
    #xtrain, ytrain = xtrain[outlierMask], ytrain[outlierMask]
    if verbose:
        print(f'train len no outliers: {len(xtrain)}')
        print(f'percent of len removed: {round(abs(len(xtrain)/lentemp*100-100),2)}%')
        print(f'test len: {len(xtest)}\n')
    
    return xtrain, ytrain, xtest, ytest

In [3]:
def matchListToDfs(df):
    matchListDateFilter = (df[df['Date'] >= pd.to_datetime('2019-7-01',format='%Y-%m-%d')]
                                        .reset_index(drop=True).copy())
    matchListDateFilter['realSemesterYear'] = (matchListDateFilter['realYear'].astype(str)
                                               +matchListDateFilter['realSemester'].astype(str))
    matchListDateFilter['tournament_id'] = (matchListDateFilter['TournamentRegion'].astype(str)
                                               +matchListDateFilter['realSemesterYear'].astype(str))
    
    playerMatchList = matchListDateFilter.copy()
    teamMatchList = matchListDateFilter.copy()

    for color in ['Blue','Red']:
        for feature in meanFeatures:
                teamMatchList[f'Team_{color}_{feature}'] = (matchListDateFilter[[f"{position}_{color}_{feature}" for position in positions]]
                                                        .mean(skipna=True,axis=1).copy())
                teamMatchList.drop([f"{position}_{color}_{feature}" for position in positions],axis=1,inplace=True)

        for feature in sumFeatures:
                teamMatchList[f'Team_{color}_{feature}'] = (matchListDateFilter[[f"{position}_{color}_{feature}" for position in positions]]
                                                        .sum(skipna=True,axis=1).copy())
                teamMatchList.drop([f"{position}_{color}_{feature}" for position in positions],axis=1,inplace=True)

        teamMatchList.drop([f"{position}_{color}" for position in positions],axis=1,inplace=True)
        
    return playerMatchList, teamMatchList

def regionLists(df, currentYear):
    regions = df['TournamentRegion'].unique()
    regionsToFeed = [x for x in df['TournamentRegion'].unique()]
    regionsFilterTemp = ([x for x in regions if currentYear in (df[df['TournamentRegion']==x])['realYear'].unique()
                                            and currentYear-1 in (df[df['TournamentRegion']==x])['realYear'].unique()])
    regionsToPredict = []
    for region in regionsFilterTemp:
        regionsFilterSize = df[(df['realYear']==currentYear) & (df['TournamentRegion']==region)]
        regionsFilterSizeTrain = df[(df['realYear']!=currentYear) & (df['TournamentRegion']==region)]
        if len(regionsFilterSize)>=30:
            regionsToPredict.append(region)
    
    return regions, regionsToFeed, regionsToPredict

def generateRegionDf(df, regionDataListF, regionsFeatureColsF, cut_off_var, tempTournamentIdF, currentTarget, splitType):
    
    dfTemp = df[df['TournamentRegion'].isin(regionDataListF)].copy()
    print(max(dfTemp['realSemesterYear']))
    tempCols = [x for x in list(dfTemp.columns) if x.split('_')[-1] in regionsFeatureColsF]
    dfTemp = dfTemp[tempCols+infoCols]
    dfTemp = dfTemp.sort_values(by='Date',ascending=True).copy()
    
    xtrain,ytrain,xtest,ytest = train_test_split_binary(dfTemp, tempTournamentIdF, currentTarget, cut_off_var, splitType, verbose=False)
    
    return dfTemp, xtrain, ytrain, xtest, ytest

def generateMetric(model_number, regionDataListF, regionsFeatureColsF, cut_off_var, tempTournamentIdF, currentTarget, dfToSplit, splitType):
    
    dfTemp, xtrain, ytrain, xtest, ytest = generateRegionDf(dfToSplit, regionDataListF, regionsFeatureColsF
                                                            , cut_off_var, tempTournamentIdF, currentTarget, splitType)
    
    print('number of 0s:')
    print(list(ytrain).count(0)/len(ytrain))
    print(ytrain.unique())
    print('len of train:')
    print(len(xtrain))
    print(len(ytrain))
    print('num of cols:')
    print(len(xtrain.columns))

    errors3=0
    rep=3
    for i in range(rep):
        print('a')
        region_model = base_models[model_number]
        region_model.fit(xtrain, ytrain)
        pred = region_model.predict(xtest)
        errors3 = accuracy_score(ytest, pred)+errors3
    errors2=errors3/rep
    
    errors3=0
    for i in range(rep):
        print('b')
        region_model = base_models[model_number]
        region_model.fit(xtrain, ytrain)
        pred = region_model.predict(xtest)
        errors3 = accuracy_score(ytest, pred)+errors3
    errors4=errors3/rep
    
    print('len of pred:')
    print(len(pred))
    print('========================')
    metric=(errors2+errors4)/2
    metric=round(abs(metric-1),3)
    
    return metric, pred

hide_toggle()

# Load

In [4]:
teamDataTable = pd.read_pickle("Data/raw_data/teamDataTable.pkl")
playerDataTable = pd.read_pickle("Data/raw_data/playerDataTable.pkl")

matchList = pd.read_pickle("Data/raw_data/matchList.pkl")
matchListFill = pd.read_pickle("Data/raw_data/matchListFill.pkl")

teamMatchList = pd.read_pickle("Data/raw_data/teamMatchList.pkl")
playerMatchList = pd.read_pickle("Data/raw_data/playerMatchList.pkl")

regionsStats = pd.read_pickle("./Data/raw_data/regionsStats.pkl")

with open(f'./Data/raw_data/regionsFeatureCols.json', 'r') as fp:
    regionsFeatureCols = json.load(fp)
with open(f'./Data/raw_data/regionsTrainData.json', 'r') as fp:
    regionsTrainData = json.load(fp)
    

In [13]:
content=1
playerMatchList, teamMatchList = matchListToDfs(matchListFill)
#             0              1
dfsContent = [playerMatchList, teamMatchList]
dfToSplit = dfsContent[content].copy()

print(f'main df size: {len(dfToSplit)}')

params = {'objective': 'binary:logistic'}
base_models = [
              RandomForestClassifier(), #0
              #XGBClassifier(params=params,num_class=2), #1
              KNeighborsClassifier(algorithm = 'brute'), #2
              LinearSVC(C=0.0001), #3
              BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5,max_features=1.0,n_estimators=10), #4
              AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=4),n_estimators=10,learning_rate=0.6), #5
              DecisionTreeClassifier(), #6
              LogisticRegression(), #7
              LogisticRegression(solver='newton-cg'), #8
              LogisticRegression(solver='newton-cg') #8
              ]

currentTarget = 'Score'
currentYear = 2022
currentSemester = 1
currentSemesterYear = str(currentYear)+str(currentSemester)
defaultModel = 8
infoCols = ['Date','tournament_id',currentTarget,'TournamentRegion']
splitType = 0

dfToSplit.drop('totalKills',axis=1,inplace=True)
featureCols = [x for x in dfToSplit.columns if x not in offCols+infoCols]
featureCols = list(set([x.replace('Team_Blue_','').replace('Team_Red_','') for x in featureCols]))

regions, regionsToFeed, regionsToPredict = regionLists(dfToSplit, currentYear)

dfToSplit = dfToSplit[dfToSplit['realSemesterYear'].astype(int)<=int(currentSemesterYear)]

regionsToPredict.remove('MSI')
regionsToPredict.remove('World')
#regionsToPredict = regionsToPredict[:5]

main df size: 26314


# SKLEARN

In [6]:
regionsStats = pd.DataFrame(columns=['region','model','size'])
regionsStats['region'] = regionsToPredict
regionsList = regionsStats['region']
regionsStats['model'] = defaultModel

regionsTrainData = dict(zip(regionsList,regionsList.apply(lambda x: [x])))
regionsStats['accuracy_0'] = np.nan
regionsStats['accuracy_1'] = np.nan
regionsStats['accuracy_2'] = np.nan
regionsStats['cut_off_var'] = 1.5

regionsFeatureCols = dict(zip(regionsList,[0]*len(regionsList)))
for key in regionsFeatureCols:
    regionsFeatureCols[key] = featureCols.copy()

### TRAIN DATA SELECTION

In [7]:
%%time

regionsTrainData = dict(zip(regionsList,regionsList.apply(lambda x: [x])))
regionsStats['accuracy_0'] = np.nan

for n,region in enumerate(regionsToPredict):
    print('=========\n')
    regionFinalAcc = regionsStats['accuracy_0'][n]
    tempTournamentId = region+currentSemesterYear
    cut_off_var = regionsStats['cut_off_var'][n]
    region_model_number = regionsStats['model'][n]
    print(f'[{n+1} of {len(regionsToPredict)}] region {region} -> {regionFinalAcc}:\n')
    
    regionsToTest = [x for x in regionsToFeed]
    regionsToTest.remove(region)
    random.shuffle(regionsToTest)
    for nn,regionToTest in enumerate(regionsToTest):
        regionsTrainData[region].append(regionToTest)
        
        metric, pred = generateMetric(region_model_number, regionsTrainData[region], regionsFeatureCols[region]
                                      , cut_off_var, tempTournamentId, currentTarget, dfToSplit, splitType)
        
        if metric < regionFinalAcc or np.isnan(regionFinalAcc):
            regionFinalAcc = metric
            
            print(f'{regionFinalAcc} -> {regionToTest} added                                           ')
        else:
            regionsTrainData[region].remove(regionToTest)
            
        #print(f'[{nn+1} of {len(regionsToTest)}] testing: {regionToTest}                        ',end='\r')
    
    regionsStats['accuracy_0'][n] = regionFinalAcc
    regionsStats['size'][n] = len(pred)
    
    print(f'\n\naccuracy: {regionFinalAcc}')
    print(f'{region} train data: {regionsTrainData[region]}\nlen:{len(regionsTrainData[region])}')
    print(f'test data len: {len(pred)}\n')
    
printFinalResults(regionsStats, 'accuracy_0')

hide_toggle()


[1 of 25] region LPL -> nan:

20221
number of 0s:
0.539497633782308
[0 1]
len of train:
2747
2747
num of cols:
22
a
a
a
b
b
b
len of pred:
297
0.32 -> World added                                           
20221
number of 0s:
0.5399280575539568
[0 1]
len of train:
2780
2780
num of cols:
22
a
a
a
b
b
b
len of pred:
297
20221
number of 0s:
0.5337530605106681
[0 1]
len of train:
2859
2859
num of cols:
22
a
a
a
b
b
b
len of pred:
297
20221
number of 0s:
0.542512908777969
[0 1]
len of train:
2905
2905
num of cols:
22
a
a
a
b
b
b
len of pred:
297
0.31 -> MSI added                                           
20221
number of 0s:
0.5420560747663551
[0 1]
len of train:
2996
2996
num of cols:
22
a
a
a
b
b
b
len of pred:
297
0.3 -> Dutch added                                           
20221
number of 0s:
0.5484054669703873
[0 1]
len of train:
3512
3512
num of cols:
22
a
a
a
b
b
b
len of pred:
297
0.29 -> PG added                                           
20221
number of 0s:
0.5446640316205533
[0

a
a
b
b
b
len of pred:
343
0.367 -> EU added                                           
20221
number of 0s:
0.5263768115942029
[1 0]
len of train:
3450
3450
num of cols:
22
a
a
a
b
b
b
len of pred:
343
20221
number of 0s:
0.5266008282892641
[1 0]
len of train:
3139
3139
num of cols:
22
a
a
a
b
b
b
len of pred:
343
0.362 -> GLL added                                           
20221
number of 0s:
0.5242548217416715
[1 0]
len of train:
3422
3422
num of cols:
22
a
a
a
b
b
b
len of pred:
343
20221
number of 0s:
0.5298756445253261
[1 0]
len of train:
3297
3297
num of cols:
22
a
a
a
b
b
b
len of pred:
343
20221
number of 0s:
0.5266309486290577
[1 0]
len of train:
3173
3173
num of cols:
22
a
a
a
b
b
b
len of pred:
343
20221
number of 0s:
0.5344910757356488
[0 1]
len of train:
4146
4146
num of cols:
22
a
a
a
b
b
b
len of pred:
343
0.356 -> LCS added                                           
20221
number of 0s:
0.5321581705574083
[0 1]
len of train:
4198
4198
num of cols:
22
a
a
a
b
b
b
len of 

a
b
b
b
len of pred:
137
20221
number of 0s:
0.5575
[0 1]
len of train:
800
800
num of cols:
22
a
a
a
b
b
b
len of pred:
137
20221
number of 0s:
0.5613540197461213
[0 1]
len of train:
709
709
num of cols:
22
a
a
a
b
b
b
len of pred:
137
20221
number of 0s:
0.5485915492957747
[0 1]
len of train:
1420
1420
num of cols:
22
a
a
a
b
b
b
len of pred:
137
20221
number of 0s:
0.547921967769296
[0 1]
len of train:
1179
1179
num of cols:
22
a
a
a
b
b
b
len of pred:
137
20221
number of 0s:
0.5111326234269119
[0 1]
len of train:
1033
1033
num of cols:
22
a
a
a
b
b
b
len of pred:
137
20221
number of 0s:
0.5380577427821522
[0 1]
len of train:
1143
1143
num of cols:
22
a
a
a
b
b
b
len of pred:
137
20221
number of 0s:
0.56975505857295
[0 1]
len of train:
939
939
num of cols:
22
a
a
a
b
b
b
len of pred:
137
0.248 -> Demacia added                                           
20221
number of 0s:
0.5569908814589666
[0 1]
len of train:
1316
1316
num of cols:
22
a
a
a
b
b
b
len of pred:
137
20221
number of 0s

a
a
b
b
b
len of pred:
161
0.248 -> BRCC added                                           
20221
number of 0s:
0.47100802854594115
[1 0 2]
len of train:
2242
2242
num of cols:
22
a
a
a
b
b
b
len of pred:
161
20221
number of 0s:
0.4987538940809969
[1 0 2]
len of train:
3210
3210
num of cols:
22
a
a
a
b
b
b
len of pred:
161
20221
number of 0s:
0.48136752136752137
[1 0 2]
len of train:
2925
2925
num of cols:
22
a
a
a
b
b
b
len of pred:
161
20221
number of 0s:
0.46879150066401065
[1 0 2]
len of train:
2259
2259
num of cols:
22
a
a
a
b
b
b
len of pred:
161
20221
number of 0s:
0.4869595058339053
[1 0 2]
len of train:
2914
2914
num of cols:
22
a
a
a
b
b
b
len of pred:
161
20221
number of 0s:
0.4759196056124384
[1 0 2]
len of train:
2637
2637
num of cols:
22
a
a
a
b
b
b
len of pred:
161
20221
number of 0s:
0.46922741161065035
[1 0 2]
len of train:
2291
2291
num of cols:
22
a
a
a
b
b
b
len of pred:
161
20221
number of 0s:
0.4720607957085382
[1 0 2]
len of train:
2237
2237
num of cols:
22
a
a
a
b

b
b
b
len of pred:
62
20221
number of 0s:
0.5401713979646492
[0 1]
len of train:
3734
3734
num of cols:
22
a
a
a
b
b
b
len of pred:
62
20221
number of 0s:
0.5427122940430925
[0 1]
len of train:
3945
3945
num of cols:
22
a
a
a
b
b
b
len of pred:
62
20221
number of 0s:
0.5462488701416089
[1 0]
len of train:
3319
3319
num of cols:
22
a
a
a
b
b
b
len of pred:
62
20221
number of 0s:
0.5498666666666666
[0 1]
len of train:
3750
3750
num of cols:
22
a
a
a
b
b
b
len of pred:
62
20221
number of 0s:
0.5458100558659218
[1 0]
len of train:
3580
3580
num of cols:
22
a
a
a
b
b
b
len of pred:
62
20221
number of 0s:
0.54421768707483
[1 0]
len of train:
3234
3234
num of cols:
22
a
a
a
b
b
b
len of pred:
62
20221
number of 0s:
0.5405174864941712
[1 0]
len of train:
3517
3517
num of cols:
22
a
a
a
b
b
b
len of pred:
62
20221
number of 0s:
0.5477481726008017
[1 0]
len of train:
4241
4241
num of cols:
22
a
a
a
b
b
b
len of pred:
62
20221
number of 0s:
0.5438057177989548
[1 0]
len of train:
3253
3253
num of 

...

b
b
b
len of pred:
74
20221
number of 0s:
0.5464071856287425
[0 1]
len of train:
1336
1336
num of cols:
22
a
a
a
b
b
b
len of pred:
74
0.311 -> LPLOL added                                           
20221
number of 0s:
0.5415282392026578
[0 1]
len of train:
1806
1806
num of cols:
22
a
a
a
b
b
b
len of pred:
74
0.297 -> CBLOL_Tier2 added                                           
20221
number of 0s:
0.5366441658630665
[0 1]
len of train:
2074
2074
num of cols:
22
a
a
a
b
b
b
len of pred:
74
20221
number of 0s:
0.5421424687330071
[0 1]
len of train:
1839
1839
num of cols:
22
a
a
a
b
b
b
len of pred:
74
20221
number of 0s:
0.5398373983739837
[0 1]
len of train:
1845
1845
num of cols:
22
a
a
a
b
b
b
len of pred:
74
20221
number of 0s:
0.5326044513861773
[0 1]
len of train:
2561
2561
num of cols:
22
a
a
a
b
b
b
len of pred:
74
20221
number of 0s:
0.5415282392026578
[0 1]
len of train:
1806
1806
num of cols:
22
a
a
a
b
b
b
len of pred:
74
20221
number of 0s:
0.5414424111948332
[0 1]
len of t

a
a
b
b
b
len of pred:
135
20221
number of 0s:
0.5264834727200318
[1 0]
len of train:
2511
2511
num of cols:
22
a
a
a
b
b
b
len of pred:
135
20221
number of 0s:
0.5168598524762908
[1 0]
len of train:
1898
1898
num of cols:
22
a
a
a
b
b
b
len of pred:
135
20221
number of 0s:
0.5105996365838885
[1 0]
len of train:
1651
1651
num of cols:
22
a
a
a
b
b
b
len of pred:
135
20221
number of 0s:
0.5052015604681405
[1 0]
len of train:
1538
1538
num of cols:
22
a
a
a
b
b
b
len of pred:
135
0.415 -> Trinity added                                           
20221
number of 0s:
0.5148955091151622
[1 0]
len of train:
2249
2249
num of cols:
22
a
a
a
b
b
b
len of pred:
135
20221
number of 0s:
0.5104581673306773
[1 0]
len of train:
2008
2008
num of cols:
22
a
a
a
b
b
b
len of pred:
135
20221
number of 0s:
0.5041217501585289
[1 0]
len of train:
1577
1577
num of cols:
22
a
a
a
b
b
b
len of pred:
135
20221
number of 0s:
0.5049833887043189
[1 0]
len of train:
1806
1806
num of cols:
22
a
a
a
b
b
b
len of pred:

a
b
b
b
len of pred:
90
20221
number of 0s:
0.5136690647482014
[1 0]
len of train:
2085
2085
num of cols:
22
a
a
a
b
b
b
len of pred:
90
20221
number of 0s:
0.5372117400419287
[1 0]
len of train:
1908
1908
num of cols:
22
a
a
a
b
b
b
len of pred:
90
20221
number of 0s:
0.5275626423690205
[1 0]
len of train:
2195
2195
num of cols:
22
a
a
a
b
b
b
len of pred:
90
20221
number of 0s:
0.5343554798409994
[1 0]
len of train:
1761
1761
num of cols:
22
a
a
a
b
b
b
len of pred:
90
20221
number of 0s:
0.5337078651685393
[1 0]
len of train:
1780
1780
num of cols:
22
a
a
a
b
b
b
len of pred:
90
20221
number of 0s:
0.5343554798409994
[1 0]
len of train:
1761
1761
num of cols:
22
a
a
a
b
b
b
len of pred:
90
20221
number of 0s:
0.5396751740139211
[1 0]
len of train:
2155
2155
num of cols:
22
a
a
a
b
b
b
len of pred:
90
20221
number of 0s:
0.5258942872397224
[1 0]
len of train:
1873
1873
num of cols:
22
a
a
a
b
b
b
len of pred:
90
20221
number of 0s:
0.5384615384615384
[1 0]
len of train:
1846
1846
num

a
a
b
b
b
len of pred:
64
20221
number of 0s:
0.47430004178854995
[1 0 2]
len of train:
2393
2393
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221
number of 0s:
0.46677672307057366
[1 0 2]
len of train:
2423
2423
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221
number of 0s:
0.46430028559771525
[1 0 2]
len of train:
2451
2451
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221
number of 0s:
0.4702702702702703
[0 1 2]
len of train:
2405
2405
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221
number of 0s:
0.46596858638743455
[1 0 2]
len of train:
2483
2483
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221
number of 0s:
0.47088186356073214
[0 1 2]
len of train:
2404
2404
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221
number of 0s:
0.47726374060893634
[1 0 2]
len of train:
2529
2529
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221
number of 0s:
0.48475016223231665
[1 0 2]
len of train:
3082
3082
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221
number of 0s:
0.47156783103168154
[1 0 2]

a
a
b
b
b
len of pred:
80
20221
number of 0s:
0.4351464435146444
[1 0 2]
len of train:
956
956
num of cols:
22
a
a
a
b
b
b
len of pred:
80
20221
number of 0s:
0.45099540581929554
[1 0 2]
len of train:
1306
1306
num of cols:
22
a
a
a
b
b
b
len of pred:
80
20221
number of 0s:
0.4491803278688525
[1 0 2]
len of train:
1220
1220
num of cols:
22
a
a
a
b
b
b
len of pred:
80
20221
number of 0s:
0.4343649946638207
[1 0 2]
len of train:
937
937
num of cols:
22
a
a
a
b
b
b
len of pred:
80
20221
number of 0s:
0.441747572815534
[1 0 2]
len of train:
1030
1030
num of cols:
22
a
a
a
b
b
b
len of pred:
80
20221
number of 0s:
0.5014442518775274
[1 0 2]
len of train:
1731
1731
num of cols:
22
a
a
a
b
b
b
len of pred:
80
20221
number of 0s:
0.4398382204246714
[1 0 2]
len of train:
989
989
num of cols:
22
a
a
a
b
b
b
len of pred:
80
20221
number of 0s:
0.42972699696663297
[1 0 2]
len of train:
989
989
num of cols:
22
a
a
a
b
b
b
len of pred:
80
20221
number of 0s:
0.42927794263105834
[1 0 2]
len of train:

In [8]:
print(np.mean(regionsStats['accuracy_0']))
print(np.mean(regionsStats['accuracy_1']))
print(np.mean(regionsStats['accuracy_2']))
regionsStats

0.27356
nan
nan


,region,model,size,accuracy_0,accuracy_1,accuracy_2,cut_off_var
0,LPL,8,297,0.273,NaN,NaN,1.5
1,LCK,8,343,0.344,NaN,NaN,1.5
2,PCS,8,137,0.241,NaN,NaN,1.5
3,VCS,8,161,0.236,NaN,NaN,1.5
4,Ultraliga,8,62,0.274,NaN,NaN,1.5
5,LLA,8,66,0.303,NaN,NaN,1.5
6,TCL,8,82,0.256,NaN,NaN,1.5
7,LFL,8,143,0.350,NaN,NaN,1.5
8,Prime,8,64,0.312,NaN,NaN,1.5
9,LJL,8,95,0.179,NaN,NaN,1.5


### FEATURE SELECTION

In [9]:
%%time

regionsStats['accuracy_1'] = np.nan
regionsFeatureCols = dict(zip(regionsList,[0]*len(regionsList)))
for key in regionsFeatureCols:
    regionsFeatureCols[key] = featureCols.copy()

for n,region in enumerate(regionsToPredict):
    print('=========\n')
    regionFinalAcc = regionsStats['accuracy_0'][n]
    tempTournamentId = region+currentSemesterYear
    cut_off_var = regionsStats['cut_off_var'][n]
    region_model_number = regionsStats['model'][n]
    print(f'[{n+1} of {len(regionsToPredict)}] region {region} -> {regionFinalAcc}:\n')
    print(f'model: {region_model_number}')
    
    initialFeatures = regionsFeatureCols[region].copy()
    for nn,feature in enumerate(initialFeatures):
        regionsFeatureCols[region].remove(feature)
        
        metric, pred = generateMetric(region_model_number, regionsTrainData[region], regionsFeatureCols[region]
                                      , cut_off_var, tempTournamentId, currentTarget, dfToSplit, splitType)
        if metric < regionFinalAcc or np.isnan(regionFinalAcc):
            regionFinalAcc = metric
            print(f'{feature} removed for {metric}                                                ')
        else:
            regionsFeatureCols[region].append(feature)
            
        print(f'[{nn+1} of {len(initialFeatures)}] testing: {feature}                        ',end='\r')
    
    regionsStats['accuracy_1'][n] = regionFinalAcc
    print(f'\n\naccuracy: {regionFinalAcc}')
    print(f'{region} feature count: {len(regionsFeatureCols[region])}')
    print(f'test data len: {len(pred)}\n')
    
printFinalResults(regionsStats, 'accuracy_1')

hide_toggle()


[1 of 25] region LPL -> 0.273:

model: 8
20221
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
22
a
a
a
b
b
b
len of pred:
297
20221 22] testing: Avg_assists                        
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
22
a
a
a
b
b
b
len of pred:
297
20221 22] testing: Avg_VWPM                        
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
20
a
a
a
b
b
b
len of pred:
297
20221 22] testing: Games                        
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
20
a
a
a
b
b
b
len of pred:
297
20221 22] testing: GD@15                        
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
22
a
a
a
b
b
b
len of pred:
297
20221 22] testing: Avg_deaths                        
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
20
a
a
a
b
b
b
len of pred:
297
20221 22] testing: KP%                       

a
b
b
b
len of pred:
343
20221f 22] testing: FB_%                        
number of 0s:
0.5348314606741573
[0 1]
len of train:
4450
4450
num of cols:
18
a
a
a
b
b
b
len of pred:
343
[22 of 22] testing: Avg_WCPM                        

accuracy: 0.329
LCK feature count: 20
test data len: 343


[3 of 25] region PCS -> 0.241:

model: 8
20221
number of 0s:
0.564922480620155
[0 1]
len of train:
1032
1032
num of cols:
22
a
a
a
b
b
b
len of pred:
137
20221 22] testing: Avg_assists                        
number of 0s:
0.564922480620155
[0 1]
len of train:
1032
1032
num of cols:
22
a
a
a
b
b
b
len of pred:
137
20221 22] testing: Avg_VWPM                        
number of 0s:
0.564922480620155
[0 1]
len of train:
1032
1032
num of cols:
20
a
a
a
b
b
b
len of pred:
137
20221 22] testing: Games                        
number of 0s:
0.564922480620155
[0 1]
len of train:
1032
1032
num of cols:
20
a
a
a
b
b
b
len of pred:
137
20221 22] testing: GD@15                        
number of 0s:
0.564922480

a
a
b
b
b
len of pred:
161
20221f 22] testing: DMG%                        
number of 0s:
0.48140635564570655
[1 0 2]
len of train:
2958
2958
num of cols:
20
a
a
a
b
b
b
len of pred:
161
20221f 22] testing: FB_%                        
number of 0s:
0.48140635564570655
[1 0 2]
len of train:
2958
2958
num of cols:
20
a
a
a
b
b
b
len of pred:
161
[22 of 22] testing: Avg_WCPM                        

accuracy: 0.224
VCS feature count: 21
test data len: 161


[5 of 25] region Ultraliga -> 0.274:

model: 8
20221
number of 0s:
0.5453061224489796
[1 0]
len of train:
3675
3675
num of cols:
22
a
a
a
b
b
b
len of pred:
62
20221 22] testing: Avg_assists                        
number of 0s:
0.5453061224489796
[1 0]
len of train:
3675
3675
num of cols:
22
a
a
a
b
b
b
len of pred:
62
20221 22] testing: Avg_VWPM                        
number of 0s:
0.5453061224489796
[1 0]
len of train:
3675
3675
num of cols:
20
a
a
a
b
b
b
len of pred:
62
20221 22] testing: Games                        
number of 

a
a
b
b
b
len of pred:
66
20221f 22] testing: Avg_WPM                        
number of 0s:
0.5305282414818202
[1 0]
len of train:
4373
4373
num of cols:
18
a
a
a
b
b
b
len of pred:
66
20221f 22] testing: DMG%                        
number of 0s:
0.5305282414818202
[1 0]
len of train:
4373
4373
num of cols:
20
a
a
a
b
b
b
len of pred:
66
20221f 22] testing: FB_%                        
number of 0s:
0.5305282414818202
[1 0]
len of train:
4373
4373
num of cols:
20
a
a
a
b
b
b
len of pred:
66
[22 of 22] testing: Avg_WCPM                        

accuracy: 0.288
LLA feature count: 21
test data len: 66


[7 of 25] region TCL -> 0.256:

model: 8
20221
number of 0s:
0.52
[0 1]
len of train:
1050
1050
num of cols:
22
a
a
a
b
b
b
len of pred:
82
20221 22] testing: Avg_assists                        
number of 0s:
0.52
[0 1]
len of train:
1050
1050
num of cols:
22
a
a
a
b
b
b
len of pred:
82
20221 22] testing: Avg_VWPM                        
number of 0s:
0.52
[0 1]
len of train:
1050
1050
nu

a
a
b
b
b
len of pred:
143
20221f 22] testing: DMG%                        
number of 0s:
0.5375751503006012
[1 0 2]
len of train:
1996
1996
num of cols:
22
a
a
a
b
b
b
len of pred:
143
20221f 22] testing: FB_%                        
number of 0s:
0.5375751503006012
[1 0 2]
len of train:
1996
1996
num of cols:
22
a
a
a
b
b
b
len of pred:
143
[22 of 22] testing: Avg_WCPM                        

accuracy: 0.35
LFL feature count: 22
test data len: 143


[9 of 25] region Prime -> 0.312:

model: 8
20221
number of 0s:
0.5286603340976089
[0 1]
len of train:
3053
3053
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221 22] testing: Avg_assists                        
number of 0s:
0.5286603340976089
[0 1]
len of train:
3053
3053
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221 22] testing: Avg_VWPM                        
number of 0s:
0.5286603340976089
[0 1]
len of train:
3053
3053
num of cols:
20
a
a
a
b
b
b
len of pred:
64
20221 22] testing: Games                        
number of 0s:
0.5

...

a
a
b
b
b
len of pred:
98
20221f 22] testing: Avg_WPM                        
number of 0s:
0.539031339031339
[1 0]
len of train:
1755
1755
num of cols:
20
a
a
a
b
b
b
len of pred:
98
20221f 22] testing: DMG%                        
number of 0s:
0.539031339031339
[1 0]
len of train:
1755
1755
num of cols:
22
a
a
a
b
b
b
len of pred:
98
20221f 22] testing: FB_%                        
number of 0s:
0.539031339031339
[1 0]
len of train:
1755
1755
num of cols:
22
a
a
a
b
b
b
len of pred:
98
[22 of 22] testing: Avg_WCPM                        

accuracy: 0.296
LEC feature count: 22
test data len: 98


[17 of 25] region CBLOL -> 0.242:

model: 8
20221
number of 0s:
0.5236108060619372
[1 0]
len of train:
4553
4553
num of cols:
22
a
a
a
b
b
b
len of pred:
91
20221 22] testing: Avg_assists                        
number of 0s:
0.5236108060619372
[1 0]
len of train:
4553
4553
num of cols:
22
a
a
a
b
b
b
len of pred:
91
20221 22] testing: Avg_VWPM                        
number of 0s:
0.5236108

len of pred:
41
20221f 22] testing: Avg_WPM                        
number of 0s:
0.4905913978494624
[0 1]
len of train:
1488
1488
num of cols:
20
a
a
a
b
b
b
len of pred:
41
20221f 22] testing: DMG%                        
number of 0s:
0.4905913978494624
[0 1]
len of train:
1488
1488
num of cols:
22
a
a
a
b
b
b
len of pred:
41
20221f 22] testing: FB_%                        
number of 0s:
0.4905913978494624
[0 1]
len of train:
1488
1488
num of cols:
22
a
a
a
b
b
b
len of pred:
41
[22 of 22] testing: Avg_WCPM                        

accuracy: 0.098
LPLOL feature count: 22
test data len: 41


[19 of 25] region LCS -> 0.322:

model: 8
20221
number of 0s:
0.5398618957940992
[1 0]
len of train:
3186
3186
num of cols:
22
a
a
a
b
b
b
len of pred:
121
20221 22] testing: Avg_assists                        
number of 0s:
0.5398618957940992
[1 0]
len of train:
3186
3186
num of cols:
22
a
a
a
b
b
b
len of pred:
121
20221 22] testing: Avg_VWPM                        
number of 0s:
0.539861895794

a
a
b
b
b
len of pred:
39
20221f 22] testing: VSPM                        
number of 0s:
0.5516039051603905
[0 1]
len of train:
4302
4302
num of cols:
18
a
a
a
b
b
b
len of pred:
39
20221f 22] testing: GPM                        
number of 0s:
0.5516039051603905
[0 1]
len of train:
4302
4302
num of cols:
20
a
a
a
b
b
b
len of pred:
39
20221f 22] testing: Avg_WPM                        
number of 0s:
0.5516039051603905
[0 1]
len of train:
4302
4302
num of cols:
18
a
a
a
b
b
b
len of pred:
39
20221f 22] testing: DMG%                        
number of 0s:
0.5516039051603905
[0 1]
len of train:
4302
4302
num of cols:
20
a
a
a
b
b
b
len of pred:
39
20221f 22] testing: FB_%                        
number of 0s:
0.5516039051603905
[0 1]
len of train:
4302
4302
num of cols:
20
a
a
a
b
b
b
len of pred:
39
[22 of 22] testing: Avg_WCPM                        

accuracy: 0.154
PG feature count: 21
test data len: 39


[21 of 25] region LCO -> 0.284:

model: 8
20221
number of 0s:
0.5534615384615384


b
len of pred:
135
20221f 22] testing: FB_Victim                        
number of 0s:
0.5052015604681405
[1 0]
len of train:
1538
1538
num of cols:
16
a
a
a
b
b
b
len of pred:
135
20221f 22] testing: VSPM                        
number of 0s:
0.5052015604681405
[1 0]
len of train:
1538
1538
num of cols:
16
a
a
a
b
b
b
len of pred:
135
20221f 22] testing: GPM                        
number of 0s:
0.5052015604681405
[1 0]
len of train:
1538
1538
num of cols:
18
a
a
a
b
b
b
len of pred:
135
20221f 22] testing: Avg_WPM                        
number of 0s:
0.5052015604681405
[1 0]
len of train:
1538
1538
num of cols:
16
a
a
a
b
b
b
len of pred:
135
20221f 22] testing: DMG%                        
number of 0s:
0.5052015604681405
[1 0]
len of train:
1538
1538
num of cols:
18
a
a
a
b
b
b
len of pred:
135
20221f 22] testing: FB_%                        
number of 0s:
0.5052015604681405
[1 0]
len of train:
1538
1538
num of cols:
18
a
a
a
b
b
b
len of pred:
135
[22 of 22] testing: Avg_WCPM    

a
a
b
b
b
len of pred:
64
20221f 22] testing: FB_Victim                        
number of 0s:
0.4818568159529258
[1 0 2]
len of train:
3059
3059
num of cols:
20
a
a
a
b
b
b
len of pred:
64
20221f 22] testing: VSPM                        
number of 0s:
0.4818568159529258
[1 0 2]
len of train:
3059
3059
num of cols:
20
a
a
a
b
b
b
len of pred:
64
20221f 22] testing: GPM                        
number of 0s:
0.4818568159529258
[1 0 2]
len of train:
3059
3059
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221f 22] testing: Avg_WPM                        
number of 0s:
0.4818568159529258
[1 0 2]
len of train:
3059
3059
num of cols:
20
a
a
a
b
b
b
len of pred:
64
20221f 22] testing: DMG%                        
number of 0s:
0.4818568159529258
[1 0 2]
len of train:
3059
3059
num of cols:
22
a
a
a
b
b
b
len of pred:
64
20221f 22] testing: FB_%                        
number of 0s:
0.4818568159529258
[1 0 2]
len of train:
3059
3059
num of cols:
22
a
a
a
b
b
b
len of pred:
64
[22 of 22] testing:

### MODEL SELECTION

In [14]:
%%time

regionsStats['accuracy_2'] = np.nan

for n,region in enumerate(regionsToPredict):
    print('=========\n')
    currAcc = regionsStats['accuracy_0'][n]
    currModel = regionsStats['model'][n]
    tempTournamentId = region+currentSemesterYear
    cut_off_var = regionsStats['cut_off_var'][n]
    print(f'[{n+1} of {len(regionsToPredict)}] region {region} -> {currAcc}:\n')
    print(f'current model: {currModel}\n')
    
    bestModelAbs = (regionsStats[regionsStats['region']==region])['accuracy_2'].iloc[0]
    for model in range(len(base_models)):
        metricModelAbs, pred = generateMetric(model, regionsTrainData[region], regionsFeatureCols[region]
                                      , cut_off_var, tempTournamentId, currentTarget, dfToSplit, splitType)
        if metricModelAbs<bestModelAbs or np.isnan(bestModelAbs):
            bestModelAbs=metricModelAbs
            bestModel=model
        print(f'model {model} -> {metricModelAbs}')

    regionsStats['model'][n] = bestModel
    regionsStats['accuracy_2'][n] = bestModelAbs
    
    print(f'\naccuracy: {bestModelAbs}')
    print(f'best model: {bestModel}\n')
    
printFinalResults(regionsStats, 'accuracy_2')

hide_toggle()


[1 of 25] region LPL -> 0.273:

current model: 8

20221
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
20
a
a
a
b
b
b
len of pred:
297
model 0 -> 0.34
20221
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
20
a
a
a
b
b
b
len of pred:
297
model 1 -> 0.532
20221
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
20
a
a
a
b
b
b
len of pred:
297
model 2 -> 0.285
20221
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
20
a
a
a
b
b
b
len of pred:
297
model 3 -> 0.373
20221
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
20
a
a
a
b
b
b
len of pred:
297
model 4 -> 0.328
20221
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
20
a
a
a
b
b
b
len of pred:
297
model 5 -> 0.34
20221
number of 0s:
0.5416946121171473
[0 1]
len of train:
4473
4473
num of cols:
20
a
a
a
b
b
b
len of pred:
297
model 6 -> 0.333
20221
number of 0s:
0.541694612117

a
a
b
b
b
len of pred:
66
model 4 -> 0.288
20221
number of 0s:
0.5305282414818202
[1 0]
len of train:
4373
4373
num of cols:
20
a
a
a
b
b
b
len of pred:
66
model 5 -> 0.452
20221
number of 0s:
0.5305282414818202
[1 0]
len of train:
4373
4373
num of cols:
20
a
a
a
b
b
b
len of pred:
66
model 6 -> 0.364
20221
number of 0s:
0.5305282414818202
[1 0]
len of train:
4373
4373
num of cols:
20
a
a
a
b
b
b
len of pred:
66
model 7 -> 0.288
20221
number of 0s:
0.5305282414818202
[1 0]
len of train:
4373
4373
num of cols:
20
a
a
a
b
b
b
len of pred:
66
model 8 -> 0.288

accuracy: 0.288
best model: 4


[7 of 25] region TCL -> 0.256:

current model: 8

20221
number of 0s:
0.52
[0 1]
len of train:
1050
1050
num of cols:
20
a
a
a
b
b
b
len of pred:
82
model 0 -> 0.315
20221
number of 0s:
0.52
[0 1]
len of train:
1050
1050
num of cols:
20
a
a
a
b
b
b
len of pred:
82
model 1 -> 0.451
20221
number of 0s:
0.52
[0 1]
len of train:
1050
1050
num of cols:
20
a
a
a
b
b
b
len of pred:
82
model 2 -> 0.366
20221


a
a
b
b
b
len of pred:
75
model 0 -> 0.362
20221
number of 0s:
0.5369504209541628
[1 0]
len of train:
3207
3207
num of cols:
22
a
a
a
b
b
b
len of pred:
75
model 1 -> 0.373
20221
number of 0s:
0.5369504209541628
[1 0]
len of train:
3207
3207
num of cols:
22
a
a
a
b
b
b
len of pred:
75
model 2 -> 0.32
20221
number of 0s:
0.5369504209541628
[1 0]
len of train:
3207
3207
num of cols:
22
a
a
a
b
b
b
len of pred:
75
model 3 -> 0.347
20221
number of 0s:
0.5369504209541628
[1 0]
len of train:
3207
3207
num of cols:
22
a
a
a
b
b
b
len of pred:
75
model 4 -> 0.351
20221
number of 0s:
0.5369504209541628
[1 0]
len of train:
3207
3207
num of cols:
22
a
a
a
b
b
b
len of pred:
75
model 5 -> 0.42
20221
number of 0s:
0.5369504209541628
[1 0]
len of train:
3207
3207
num of cols:
22
a
a
a
b
b
b
len of pred:
75
model 6 -> 0.333
20221
number of 0s:
0.5369504209541628
[1 0]
len of train:
3207
3207
num of cols:
22
a
a
a
b
b
b
len of pred:
75
model 7 -> 0.213
20221
number of 0s:
0.5369504209541628
[1 0]
len 

b
b
len of pred:
91
model 5 -> 0.375
20221
number of 0s:
0.5236108060619372
[1 0]
len of train:
4553
4553
num of cols:
22
a
a
a
b
b
b
len of pred:
91
model 6 -> 0.352
20221
number of 0s:
0.5236108060619372
[1 0]
len of train:
4553
4553
num of cols:
22
a
a
a
b
b
b
len of pred:
91
model 7 -> 0.242
20221
number of 0s:
0.5236108060619372
[1 0]
len of train:
4553
4553
num of cols:
22
a
a
a
b
b
b
len of pred:
91
model 8 -> 0.242

accuracy: 0.242
best model: 7


[18 of 25] region LPLOL -> 0.098:

current model: 8

20221
number of 0s:
0.4905913978494624
[0 1]
len of train:
1488
1488
num of cols:
22
a
a
a
b
b
b
len of pred:
41
model 0 -> 0.154
20221
number of 0s:
0.4905913978494624
[0 1]
len of train:
1488
1488
num of cols:
22
a
a
a
b
b
b
len of pred:
41
model 1 -> 0.171
20221
number of 0s:
0.4905913978494624
[0 1]
len of train:
1488
1488
num of cols:
22
a
a
a
b
b
b
len of pred:
41
model 2 -> 0.118
20221
number of 0s:
0.4905913978494624
[0 1]
len of train:
1488
1488
num of cols:
22
a
a
a
b
b
b


a
a
b
b
b
len of pred:
90
model 2 -> 0.222
20221
number of 0s:
0.5343554798409994
[1 0]
len of train:
1761
1761
num of cols:
22
a
a
a
b
b
b
len of pred:
90
model 3 -> 0.317
20221
number of 0s:
0.5343554798409994
[1 0]
len of train:
1761
1761
num of cols:
22
a
a
a
b
b
b
len of pred:
90
model 4 -> 0.4
20221
number of 0s:
0.5343554798409994
[1 0]
len of train:
1761
1761
num of cols:
22
a
a
a
b
b
b
len of pred:
90
model 5 -> 0.461
20221
number of 0s:
0.5343554798409994
[1 0]
len of train:
1761
1761
num of cols:
22
a
a
a
b
b
b
len of pred:
90
model 6 -> 0.222
20221
number of 0s:
0.5343554798409994
[1 0]
len of train:
1761
1761
num of cols:
22
a
a
a
b
b
b
len of pred:
90
model 7 -> 0.222
20221
number of 0s:
0.5343554798409994
[1 0]
len of train:
1761
1761
num of cols:
22
a
a
a
b
b
b
len of pred:
90
model 8 -> 0.222

accuracy: 0.222
best model: 2


[24 of 25] region LVP -> 0.312:

current model: 8

20221
number of 0s:
0.4818568159529258
[1 0 2]
len of train:
3059
3059
num of cols:
22
a
a
a
b


In [15]:
print(np.mean(regionsStats['accuracy_0']))
print(np.mean(regionsStats['accuracy_1']))
print(np.mean(regionsStats['accuracy_2']))
regionsStats

0.27356
0.26055999999999996
0.25611999999999996


,region,model,size,accuracy_0,accuracy_1,accuracy_2,cut_off_var
0,LPL,7,297,0.273,0.256,0.256,1.5
1,LCK,7,343,0.344,0.329,0.329,1.5
2,PCS,7,137,0.241,0.241,0.241,1.5
3,VCS,7,161,0.236,0.224,0.224,1.5
4,Ultraliga,7,62,0.274,0.258,0.258,1.5
5,LLA,4,66,0.303,0.288,0.288,1.5
6,TCL,7,82,0.256,0.244,0.244,1.5
7,LFL,7,143,0.350,0.350,0.350,1.5
8,Prime,4,64,0.312,0.312,0.234,1.5
9,LJL,7,95,0.179,0.179,0.179,1.5


In [34]:
print(np.mean(regionsStats['accuracy_0']))
print(np.mean(regionsStats['accuracy_1']))
print(np.mean(regionsStats['accuracy_2']))
regionsStats

0.26983999999999997
0.26024
0.262


,region,model,size,accuracy_0,accuracy_1,accuracy_2,cut_off_var
0,LPL,3,297,0.290,0.279,0.279,1.5
1,LCK,8,343,0.315,0.315,0.315,1.5
2,PCS,8,137,0.212,0.212,0.212,1.5
3,VCS,8,161,0.248,0.242,0.242,1.5
4,Ultraliga,8,62,0.161,0.161,NaN,1.5
5,LLA,8,66,0.303,0.303,NaN,1.5
6,TCL,8,82,0.256,0.256,NaN,1.5
7,LFL,8,143,0.378,0.378,NaN,1.5
8,Prime,8,64,0.328,0.250,NaN,1.5
9,LJL,8,95,0.189,0.189,NaN,1.5


# Notes

In [8]:
### DROP OUTLIERS

%%time

# regionsFeatureCols = dict(zip(regionsList,[0]*len(regionsList)))
# for key in regionsFeatureCols:
#     regionsFeatureCols[key] = featureCols.copy()

# for n,region in enumerate(regionsToPredict):
#     print('=========\n')
#     regionFinalAcc = regionsStats['accuracy_0'][n]
#     tempTournamentId = region+currentSemesterYear
#     cut_off_var = regionsStats['cut_off_var'][n]
#     region_model_number = regionsStats['model'][n]
#     print(f'[{n+1} of {len(regionsToPredict)}] region {region} -> {regionFinalAcc}:\n')
#     print(f'current var: {cut_off_var}')
#     for var in np.arange(1.0,2.0,0.1):
        
#         metric, pred = generateMetric(region_model_number, regionsTrainData[region], regionsFeatureCols[region]
#                                       , var, tempTournamentId, currentTarget, dfToSplit, splitType)
        
#         print(f'var: {round(var,2)}, metric: {metric}')
#         if metric < regionFinalAcc or np.isnan(regionFinalAcc):
#             regionFinalAcc = metric
#             regionsStats['cut_off_var'][n] = var
#             print(f'changed to {round(var,2)} cut-off for {metric}                                                ')
            
#         #print(f'[{nn+1} of {len(initialFeatures)}] testing: {feature}                        ',end='\r')
    
#     regionsStats['accuracy_0'][n] = regionFinalAcc
#     print(f'\n\naccuracy: {regionFinalAcc}')
#     print(f'test data len: {len(pred)}\n')
    
# printFinalResults(regionsStats, 'accuracy_0')

hide_toggle()


[1 of 27] region LPL -> 0.332:

current var: 1.5
var: 1.0, metric: 0.428
var: 1.1, metric: 0.351
var: 1.2, metric: 0.37
var: 1.3, metric: 0.358
var: 1.4, metric: 0.37
var: 1.5, metric: 0.347
var: 1.6, metric: 0.373
var: 1.7, metric: 0.354
var: 1.8, metric: 0.34
var: 1.9, metric: 0.384


accuracy: 0.332
test data len: 297


[2 of 27] region LCK -> 0.372:

current var: 1.5
var: 1.0, metric: 0.39
var: 1.1, metric: 0.408
var: 1.2, metric: 0.405
var: 1.3, metric: 0.386
var: 1.4, metric: 0.404
var: 1.5, metric: 0.405
var: 1.6, metric: 0.416
var: 1.7, metric: 0.379
var: 1.8, metric: 0.401
var: 1.9, metric: 0.4


accuracy: 0.372
test data len: 343


[3 of 27] region PCS -> 0.246:

current var: 1.5
var: 1.0, metric: 0.336
var: 1.1, metric: 0.258
var: 1.2, metric: 0.276
var: 1.3, metric: 0.266
var: 1.4, metric: 0.277
var: 1.5, metric: 0.263
var: 1.6, metric: 0.283
var: 1.7, metric: 0.248
var: 1.8, metric: 0.273
var: 1.9, metric: 0.27


accuracy: 0.246
test data len: 137


[4 of 27] region VCS -

var: 1.2, metric: 0.287
var: 1.3, metric: 0.265
var: 1.4, metric: 0.243
var: 1.5, metric: 0.246
var: 1.6, metric: 0.283
var: 1.7, metric: 0.281
var: 1.8, metric: 0.276
var: 1.9, metric: 0.261


accuracy: 0.226
test data len: 90


[25 of 27] region LVP -> 0.448:

current var: 1.5
var: 1.0, metric: 0.688
var: 1.1, metric: 0.688
var: 1.2, metric: 0.5
var: 1.3, metric: 0.508
var: 1.4, metric: 0.544
var: 1.5, metric: 0.518
var: 1.6, metric: 0.536
var: 1.7, metric: 0.492
var: 1.8, metric: 0.508
var: 1.9, metric: 0.526


accuracy: 0.448
test data len: 64


[26 of 27] region NA_Tier2 -> 0.812:

current var: 1.5
var: 1.0, metric: 0.896
var: 1.1, metric: 0.904
var: 1.2, metric: 0.892
var: 1.3, metric: 0.9
var: 1.4, metric: 0.892
var: 1.5, metric: 0.892
var: 1.6, metric: 0.879
var: 1.7, metric: 0.875
var: 1.8, metric: 0.946
var: 1.9, metric: 0.862


accuracy: 0.812
test data len: 80


[27 of 27] region MSI -> nan:

current var: 1.5


ValueError: Found array with 0 sample(s) (shape=(0, 44)) while a minimum of 1 is required by BaggingClassifier.